In [22]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
import matplotlib.pyplot as plt

In [27]:
COIN_NAME = "Bitcoin"
HORIZON = "1h"
TIMESTAMP = "2024-10-15 05:00:00" 

BASE_DIR = "/Users/ayushgupta/Desktop/ML-Driven-Web-Platform-for-Cryptocurrency-Price-Forecasting_November_Batch-5_2025"

MODEL_PATH = f"{BASE_DIR}/Milestone_2/models/{COIN_NAME.lower()}/{HORIZON}/final_lstm_{COIN_NAME}_{HORIZON}.keras"

FEATURE_SCALER_PATH = f"{BASE_DIR}/Milestone_1/data/scaled/{COIN_NAME}/{COIN_NAME}_feature_scaler.pkl"
PRICE_SCALER_PATH   = f"{BASE_DIR}/Milestone_1/data/scaled/{COIN_NAME}/{COIN_NAME}_price_scaler.pkl"

CSV_PATH = f"{BASE_DIR}/Milestone_1/data/processed/{COIN_NAME}_processed.csv"
SEQ_LEN = 48

# Load Dataframe

In [24]:
feature_cols = [
    "open","high","low","close","volume",
    "return_1h","volatility_24h",
    "ma_24","ma_168","ma_ratio",
    "vol_change","missing_flag"
]

# Loading Model 

In [25]:
print("\nLoading model...")
model = tf.keras.models.load_model(MODEL_PATH)

print("Loading scalers...")
feature_scaler = joblib.load(FEATURE_SCALER_PATH)
price_scaler   = joblib.load(PRICE_SCALER_PATH)


Loading model...
Loading scalers...


In [28]:
df = pd.read_csv(CSV_PATH, parse_dates=["open_time"])
df = df.set_index("open_time")

if TIMESTAMP not in df.index:
    raise ValueError(f"Timestamp {TIMESTAMP} not found in dataset.")

In [29]:
idx = df.index.get_loc(TIMESTAMP)

# Check if enough history exists
if idx < SEQ_LEN:
    raise ValueError(f"Not enough previous data before {TIMESTAMP} to create sequence.")
# Sequence window index
window = df[feature_cols].iloc[idx-SEQ_LEN:idx].values.reshape(1, SEQ_LEN, len(feature_cols))

# APPLY FEATURE SCALING
window_scaled = window.copy()
window_scaled = feature_scaler.transform(window_scaled.reshape(-1, len(feature_cols))).reshape(1, SEQ_LEN, len(feature_cols))


/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


# Inverse Predictions

In [30]:
scaled_pred = model.predict(window_scaled)[0][0]

# place pred in dummy row → inverse-transform only "close"
dummy = np.zeros((1, len(feature_cols)))
dummy[0][feature_cols.index("close")] = scaled_pred
predicted_price = scaler.inverse_transform(dummy)[0][feature_cols.index("close")]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step


2025-12-08 01:39:31.573234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


NameError: name 'scaler' is not defined

# Inverse Actual

In [31]:
actual_scaled = df["close"].loc[TIMESTAMP_TO_CHECK]
dummy2 = np.zeros((1, len(feature_cols)))
dummy2[0][feature_cols.index("close")] = actual_scaled
actual_price = scaler.inverse_transform(dummy2)[0][feature_cols.index("close")]


NameError: name 'scaler' is not defined

In [32]:
print("\n========== PREDICTION RESULT ==========")
print(f"Coin: {COIN_NAME}")
print(f"Horizon: {HORIZON}h ahead")
print(f"Timestamp: {TIMESTAMP_TO_CHECK}")
print(f"Predicted Close Price → {predicted_price:.2f}")
print(f"Actual Close Price    → {actual_price:.2f}")
print("========================================\n")



========== PREDICTION RESULT ==========
Coin: Bitcoin
Horizon: 1hh ahead
Timestamp: 2023-06-10 14:00:00


NameError: name 'predicted_price' is not defined

In [33]:
plt.figure(figsize=(7,5))
plt.bar(["Actual"], [actual_price], color="green", alpha=0.6)
plt.bar(["Predicted"], [predicted_price], color="red", alpha=0.6)
plt.title(f"{COIN_NAME} | {HORIZON}h Ahead Prediction @ {TIMESTAMP_TO_CHECK}")
plt.ylabel("Price (USD)")
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

NameError: name 'actual_price' is not defined

<Figure size 700x500 with 0 Axes>

In [38]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf

# ================================
# USER INPUTS
# ================================
COIN = "Bitcoin"
HORIZON = "1h"   # 1h, 24h etc.

BASE = "/Users/ayushgupta/Desktop/ML-Driven-Web-Platform-for-Cryptocurrency-Price-Forecasting_November_Batch-5_2025"

MODEL_PATH = f"{BASE}/Milestone_2/models/{COIN}/{HORIZON}/final_lstm_{COIN}_{HORIZON}.keras"
PRICE_SCALER_PATH = f"{BASE}/Milestone_1/data/scaled/{COIN}/{COIN}_price_scaler.pkl"
FEATURE_SCALER_PATH = f"{BASE}/Milestone_1/data/scaled/{COIN}/{COIN}_feature_scaler.pkl"
CSV_PATH = f"{BASE}/Milestone_1/data/processed/{COIN}_processed.csv"

SEQ_LEN = 48

# ================================
# LOAD MODEL + SCALERS
# ================================
print("Loading model...")
model = tf.keras.models.load_model(MODEL_PATH)

print("Loading scalers...")
feature_scaler = joblib.load(FEATURE_SCALER_PATH)
price_scaler = joblib.load(PRICE_SCALER_PATH)

# ================================
# LOAD PROCESSED DATA
# ================================
df = pd.read_csv(CSV_PATH, parse_dates=["open_time"])
df = df.set_index("open_time")

feature_cols = [
    "open","high","low","close","volume",
    "return_1h","volatility_24h",
    "ma_24","ma_168","ma_ratio",
    "vol_change","missing_flag"
]

# ================================
# BUILD LAST SEQUENCE
# ================================
last_seq = df[feature_cols].iloc[-SEQ_LEN:].values
last_seq = last_seq.reshape(1, SEQ_LEN, len(feature_cols))

# ================================
# PREDICT (SCALED)
# ================================
scaled_pred = model.predict(last_seq)[0][0]

# ================================
# INVERSE SCALE WITH price_scaler
# ================================
actual_pred = price_scaler.inverse_transform([[scaled_pred]])[0][0]

# ================================
# GET LAST REAL CLOSE PRICE
# ================================
last_close_scaled = df["close"].iloc[-1]
last_close_real = price_scaler.inverse_transform([[last_close_scaled]])[0][0]

# ================================
# PRINT RESULT
# ================================
print("\n================ PREDICTION =================")
print(f"{COIN.upper()} {HORIZON} ahead forecast:")
print(f"Predicted Close Price: {actual_pred:.2f}")
print("=============================================\n")

print("============= LAST KNOWN PRICE ==============")
print(f"Last Real Close: {last_close_real:.2f}")
print("=============================================\n")

Loading model...
Loading scalers...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step

================ PREDICTION =================
BITCOIN 1h ahead forecast:
Predicted Close Price: 0.91

============= LAST KNOWN PRICE ==============
Last Real Close: 91517.82

